In [1]:
import spacy
import json

nlp = spacy.load("en_core_web_sm")


In [2]:
with open("data.json") as f:
    data = json.load(f)

In [4]:
data[0]

{'level': 'Entry Level',
 'description': "OUTFITTERY is Europe’s biggest Personal Shopping Service and we have a clear mission: We build the future of menswear shopping. Our goal is to make men happy!\nAre you interested in building products used by hundreds of thousend of people? Do you like delivering code at a rapid pace? Outfittery is seeking an experienced Frontend Engineer (m/f) that is passionate about building mobile and desktop web applications. This position is full-time and is based in our Berlin office.\n \nYour responsibilities:\n\n\nImplement the features and user interfaces of Outfittery products like the Outfit Recommender\n\n\nArchitect efficient and reusable front-end systems that drive complex web applications\n\n\nCollaborate with Product Designers, Product Managers, and Software Engineers to deliver compelling user-facing products\n\n\nIdentify and resolve performance and scalability issues\n\n\n \nYour profile:\n\n\n3+ years of JavaScript experience, including con

In [10]:
embeddings = []
for dic in data:
    level = dic["level"] if "level" in dic.keys() else None
    title = dic["title"] if "title" in dic.keys() else None
    doc = nlp(dic["description"])
    title
    embeddings.append([level, title, doc.vector])

In [11]:
embeddings

[['Entry Level',
  'Frontend Engineer (m/f)',
  array([-0.06985028, -0.16447067, -0.2055225 ,  0.04123456,  0.15656285,
          0.35479012,  0.30583754,  0.36857107,  0.21695788, -0.07687789,
         -0.05280695,  0.00536465, -0.3993671 , -0.15417135, -0.08534644,
          0.00372916,  0.03200232,  0.13694769, -0.22489032, -0.11952426,
         -0.14080803,  0.22783725,  0.02982019, -0.10934812,  0.057911  ,
         -0.1342012 ,  0.40479118,  0.22024822,  0.25641373, -0.10261051,
         -0.00469595, -0.17551881,  0.24953668, -0.14290991,  0.18353695,
         -0.10930878,  0.50206923,  0.0706281 , -0.20413771, -0.17580517,
         -0.2986951 ,  0.17606413, -0.10680257,  0.00537238,  0.03855364,
          0.00278362, -0.17625192,  0.13042249,  0.03064273, -0.0527813 ,
         -0.33947045,  0.01088314,  0.05900292, -0.3232903 , -0.26580054,
         -0.01511579, -0.20101538, -0.08309133,  0.03895557, -0.08267199,
         -0.12144693, -0.1813356 ,  0.12978254, -0.17632462,  0.19